## Monte Carlo Simulations

In this project we will do:

- Price European call/put options
- Implement antithetic variates, control variates, and stratified sampling techniques to reduce the variance
- Use the Quasi-Monte Carlo midpoint rule to show the linear convergence rate
- Measure the absolute error using analytical Black-Scholes prices


### 1. Black-Scholes Framework (Risk-Neutral Valuation)

The price of a European derivative under the risk-neutral measure is given by:

$$V_0 = e^{-rT} \mathbb{E}^{\mathbb{Q}}[f(S_T)]$$

Where:
- $V_0$ is the present value of the option
- $f(S_T)$ is the payoff at maturity
- $r$ is the risk-free rate
- $S_T$ is the asset price at time $T$
- $\mathbb{Q}$ is the risk-neutral measure

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd

# Define parameters

S0 = 100      # Initial stock price
X = 100       # eXercise price
T = 1.0       # Time to maturity (in years)
r = 0.05      # Risk-free interest rate
sigma = 0.2   # Volatility
N_values = [int(1.1 ** i) for i in range(100, 1000, 1)]    # Number of Monte Carlo simulations
call_or_put = "call"

Under the Black-Scholes model, the stock price $S_t$ follows **Geometric Brownian Motion (GBM)**:

$$dS_t = r S_t dt + \sigma S_t dW_t^{\mathbb{Q}}$$

Solution of this SDE:

$$S_T = S_0 \cdot \exp\left( \left( r - \frac{1}{2}\sigma^2 \right) T + \sigma \sqrt{T} Z \right), \quad Z \sim \mathcal{N}(0, 1)$$

### 2. Monte Carlo Estimation

Monte Carlo approximation of the expectation:

$$\mathbb{E}[f(S_T)] \approx \frac{1}{N} \sum_{i=1}^N f(S_T^{(i)})$$

Thus, the option price becomes:

$$V_0 = e^{-rT} \cdot \mathbb{E}[f(S_T)] \approx e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^N f(S_T^{(i)})$$

Payoffs:
- Call option: $f(S_T) = \max(S_T - X, 0)$
- Put option: $f(S_T) = \max(X - S_T, 0)$

In [3]:
def standard_mc(N, S0, X, T, r, sigma, which="call"):

    Z = np.random.randn(N)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if which == "call":
        discounted_payoffs = np.exp(-r * T) * np.maximum(ST - X, 0)
    elif which == "put":
        discounted_payoffs = np.exp(-r * T) * np.maximum(X - ST, 0)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    mc_price = np.mean(discounted_payoffs)

    sample_std_dev = np.std(discounted_payoffs, ddof=1)
    standard_error = sample_std_dev / np.sqrt(N)
    standard_error_95 = 1.96 * standard_error # Calculate standard error at 95% confidence level

    return mc_price, standard_error_95

### 3. Variance Reduction Techniques

As we know, standard Monte Carlo converges at rate $O(1/\sqrt{N})$

Reducing the variance of your estimator improves accuracy per simulation

#### 3.1 Antithetic Variates

Idea: Use negatively correlated samples $Z$ and $-Z$ to reduce variance.

$$\overline{f}(Z_i) = \left(f(Z_i) + f(-Z_i)\right)$$

New estimator is still unbiased:

$$ \mathbb{E}[\overline{f}] = \frac{1}{2} \left( \mathbb{E}[f(Z)] + \mathbb{E}[f(-Z)] \right) = \mathbb{E}[f(Z)] $$

And the new variance is:
$$
\begin{align*}
\mathbb{V}[\overline{f}] = & \frac{1}{4} \left( \mathbb{V}[f(Z)] + 2 \text{Cov}[f(Z), f(−Z)] + \mathbb{V}[f(-Z)] \right) \\
                         =\ & \frac{1}{2} \left( \mathbb{V}[f(Z)] + \text{Cov}[f(Z), f(−Z)]\right)
\end{align*}
$$

- in worst-case scenario, $\text{Cov}[f(Z), f(−Z)] = \mathbb{V}[f(Z)]$, which gives no gain
- for example, in case of a linear payoff $ f = a + b Z$, $\text{Cov}[f(Z), f(−Z)] = -\mathbb{V}[f(Z)]$, which means that $\mathbb{V}[\overline{f}]$ vanishes

Effect: The fluctuations from $Z$ and $-Z$ cancel out to some degree.

In [5]:
def antithetic_variates_mc(N, S0, X, T, r, sigma, which="call"):
    Z = np.random.randn(N // 2)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    ST_anti = S0 * np.exp((r - 0.5 * sigma**2) * T - sigma * np.sqrt(T) * Z)

    if which == "call":
        discounted_payoffs = np.exp(-r * T) * 0.5 * (np.maximum(ST - X, 0) + np.maximum(ST_anti - X, 0))
    elif which == "put":
        discounted_payoffs = np.exp(-r * T) * 0.5 * (np.maximum(X - ST, 0) + np.maximum(X - ST_anti, 0))
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    mc_price = np.mean(discounted_payoffs)

    sample_std_dev = np.std(discounted_payoffs, ddof=1)
    standard_error = sample_std_dev / np.sqrt(N // 2)
    standard_error_95 = 1.96 * standard_error # Calculate standard error at 95% confidence level

    return mc_price, standard_error_95

#### 3.2 Control Variates

**Idea**: Use a variable Y correlated with the payoff $f$, where $\mathbb{E}[g] = \mu_g$ is known.

Adjusted estimator:

$$f_{\text{adj}} = f + \lambda(\mu_g - g)$$

New estimator is still unbiased:

$$ \mathbb{E}[f_{\text{adj}}] = \mathbb{E}[f] + \mathbb{E}[\lambda \mu_g - g] = \mathbb{E}[f] $$

The variance for new estimator is:

$$ \mathbb{V}[f − \lambda (\mu_g − \mathbb{E}[g])] = N^{-1} \left( \mathbb{V}[f]− 2 \lambda \text{Cov}[f, g] + \lambda^2 \mathbb{V}[g] \right)$$

Minimum is achieved for:

$$\lambda = \frac{\text{Cov}(f, g)}{\mathbb{V}(g)}$$

In this context:
- $f = \max(S_T - X, 0)$ for Call or $f = \max(X - S_T, 0)$ for Put
- $g = S_T$, with $\mathbb{E}[S_T] = S_0 e^{rT}$

Or, use the exact Black-Scholes price as control for additional adjustment.

In [6]:
def control_variates_mc(N, S0, X, T, r, sigma, which="call"):

    Z = np.random.randn(N)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    # Step 2: Calculate call option payoffs (X) and control variable (Y = S_T)

    if which == "call":
        payoffs = np.maximum(ST - X, 0)
    elif which == "put":
        payoffs = np.maximum(X - ST, 0)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    g = ST
    g_mean = S0 * np.exp(r * T)  # known expected value of S_T under risk-neutral measure

    # Step 3: Estimate lambda*
    cov = np.cov(payoffs, g, ddof=1)[0, 1]
    var_g = np.var(g, ddof=1)
    lambda_star = cov / var_g

    # Step 4: Apply control variate adjustment
    payoffs_adj = payoffs + lambda_star * (g_mean - g)
    discounted_payoffs_adj = np.exp(-r * T) * payoffs_adj

    # Step 5: Estimate option price using the adjusted estimator
    mc_price = np.mean(discounted_payoffs_adj)

    standard_error = np.std(discounted_payoffs_adj, ddof=1) / np.sqrt(N)
    standard_error_95 = 1.96 * standard_error # Calculate standard error at 95% confidence level

    return mc_price, standard_error_95

#### 3.3 Stratified Sampling

Idea: Divide the interval $[0, 1]$ into $M$ equal subintervals and sample once per interval.

Then use inverse transform sampling:

$$Z_i = \Phi^{-1}\left(\frac{i - 0.5}{M}\right), \quad i = 1, …, M$$

Where $\Phi^{-1}$ is the inverse CDF of standard normal.

The number of samples in each subinterval is $L = N/M$

In this case 

$$\mathbb{E}[\overline{F}] = M^{-1} \sum_j \mu_j, \quad \text{where} \quad \mu_j = \mathbb{E}[f(U) \; | \; U \in \text{strata} \; j],$$

and

$$\mathbb{V}[\overline{F}] = N^{-1} M^{-1} \sum_j \sigma_j^2, \quad \text{where} \quad \sigma^2_j = \mathbb{V}[f(U) \; | \; U \in \text{strata} \; j]$$

with $N = LM$ being the total number of samplings

In [7]:
def stratified_sampling_mc(N, S0, X, T, r, sigma, which="call", M=1000):

    L = N // M  # Number of samples per stratum
    ave = 0.0
    var = 0.0

    for m in range(1, M + 1):
        # Stratified uniform samples in [(m-1)/M, m/M]
        U = (m - 1 + np.random.rand(L)) / M
        Z = norm.ppf(U)

        # Simulate asset prices
        ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

        if which == "call":
            discounted_payoffs = np.exp(-r * T) * np.maximum(ST - X, 0)
        elif which == "put":
            discounted_payoffs = np.exp(-r * T) * np.maximum(X - ST, 0)
        else:
            raise ValueError("Invalid option type. Use 'call' or 'put'.")

        ave1 = np.mean(discounted_payoffs)
        var1 = np.var(discounted_payoffs, ddof=1) / L

        # Aggregate the results
        ave += ave1 / M
        var += var1 / M**2

    mc_price = ave
    standard_error_95 = 1.96 * np.sqrt(var) # Calculate standard error at 95% confidence level

    return mc_price, standard_error_95

### 4. Quasi Monte Carlo (QMC)

**Quasi Monte Carlo (QMC)** replaces random points with deterministic, low-discrepancy sequences that fill the space more uniformly:

$$
\mathbb{E}[f(X)] \approx \frac{1}{N} \sum_{i=1}^N f(\Phi^{-1}(u_i)), \quad u_i \in [0,1] \text{ (evenly spaced)}
$$

QMC can converge at a rate of $\mathcal{O}(1/N)$ under smoothness conditions.

Even sampling ensures that the domain of the random variable is uniformly covered, which prevents clustering in regions like the tails or the center, and helps stabilize the mean estimate. The variance remains unchanged in this case.

#### 4.1 The Midpoint Rule in 1D

In one dimension, a simple QMC technique is the **midpoint rule**:

$$
u_i = \frac{i - 0.5}{N}, \quad i = 1, \dots, N
$$

This gives evenly spaced points in the unit interval, which are then transformed using the **inverse CDF (PPF)** to match the target distribution, which is the standard normal distribution for Black-Scholes pricing:

$$
Z_i = \Phi^{-1}(u_i), \quad Z_i \sim \mathcal{N}(0, 1)
$$

This approach is deterministic and achieves low absolute error in 1D cases such as pricing European options. In higher dimensions one can use **Sobol** or **Halton** sequences.


In [8]:
def midpoint_rule_qmc(N, S0, X, T, r, sigma, which="call"):

    U = (np.arange(N) + 0.5) / N
    Z = norm.ppf(U)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if which == "call":
        discounted_payoffs = np.exp(-r * T) * np.maximum(ST - X, 0)
    elif which == "put":
        discounted_payoffs = np.exp(-r * T) * np.maximum(X - ST, 0)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    mc_price = np.mean(discounted_payoffs)

    return mc_price

### 5. Analytical solution to Black-Scholes equation for European options


For a **European Call**, the analytical price is:

$$C(S_0, X, T, r, \sigma) = S_0 N(d_1) - X e^{-rT} N(d_2)$$

Where:

$$d_1 = \frac{\ln(S_0 / X) + (r + \sigma^2 / 2) T}{\sigma \sqrt{T}}, \quad d_2 = d_1 - \sigma \sqrt{T}$$

**European Put** has the following form:

$$P(S_0, X, T, r, \sigma) = -S_0 N(-d_1) + X e^{-rT} N(-d_2)$$

We will use this as ground truth to measure:
- Absolute Error
- Convergence trends

In [ ]:
def black_scholes_european_options(S0, X, T, r, sigma, which="call"):

    d1 = (np.log(S0 / X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if which == "call":
        price = S0 * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    elif which == "put":
        price = X * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return price

### 6. Results

In this example we estimated the prices of European Call and Put options using the standard Monte Carlo method, several variance reduction techniques and the Quasi-Monte Carlo midpoint rule. 

Below you can find the absolute error of the estimated option price as a function of $N$ - the total number of samples. 

Conclusions:
- The **antithetic variates** method provided a variance reduction by a factor of approximately $1.5$ (almost invisible in log scale)
- The **control variates** technique achieved a gain of around a factor of $3$ compared to standard MC
- the most powerful MC-base variance reduction method is the **stratified sampling** approach, giving an improvement of up to $2$ orders of magnitude in accuracy
- The convergence rate for all MC-based methods was confirmed to follow the expected $\mathcal{O}(1/\sqrt{N})$
- In case of **Quasi-Monte Carlo** method, the convergence rate is found to be linear $\mathcal{O}(1/N)$, which is consistent with theoretical predictions

<p align="center">
  <img src="figures/mc_results_abs_err_call.svg" width="1000">
</p>

<p align="center">
  <img src="figures/mc_results_abs_err_put.svg" width="1000">
</p>

Below you can see the standard error of Monte Carlo sampling as a function of $N$, the total number of samples.

<p align="center">
  <img src="figures/mc_results_std_err_call.svg" width="1000">
</p>

<p align="center">
  <img src="figures/mc_results_std_err_put.svg" width="1000">
</p>